# **CROP YIELD ANALYSIS**
For the purpose of this project, ten (10) crops namely: **cassava**, **maize**, **plantain**, **potatoes**, **rice paddy**, **sorghum**, **soybeans**, **sweet potatoes**, **wheat** and **yam** as well as six (6) random countries namely: **Canada**, **Mexico**, **Kenya**, **Ghana**, **India** and **Spain** were selected within a period of 2010 and 2013 with a view to answering the following questions:
- What is the total yield of all crops 10 crops from 2010 to 2013?
- What is the total yield of all crops planted in Canada between 2010 and 2013?
- What is the quantity of total pesticides used between 2010 and 2013
- What is the total counts of Weather-Index Insurance claims and which countries and crops have the highest number of WI insurance claims between 2010 and 2013?

## **Data Source**
Data used for this project was sourced from (https://www.kaggle.com/datasets/patelris/crop-yield-prediction-dataset/data) which aims to analyze and predict crop yield based on various influencing factors such as rainfall and pesticides. The dataset included the following data types:
- Geographical information (Area)
- Climate and Environmental Factors (Rainfall, temperature)
- Agricultural input (Pesticides)
- Crop information (Crop type, crop yield)

## **Steps to Mart Creation**
This mart was created based on the principles of Star Schema Design as outline by [Ralph Kinball] (https://www.kimballgroup.com/data-warehouse-business-intelligence-resources/kimball-techniques/dimensional-modeling-techniques/). The following is performed in T-SQL against a SQL Server instance. MS SQL Server 2022 was used for this project. 

### **1. Schema Creation**
The following schemas were created:
-  dim
-  stg
-  fact

#### Code example using the "insert code" method
```sql
/*********************************************************************/
/****************    SCHEMA DDL      ********************************/
/*******************************************************************/
IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'dim' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA dim AUTHORIZATION dbo;'
END
;

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'stg' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA stg AUTHORIZATION dbo;'
END
;

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'f' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA f AUTHORIZATION dbo;'
END
;
GO
```
### **Explanation of Schema Creation**
- **Step 1:** Checking for the existence of the schema named "dim" in the database
- **Step 2:** If the schema does not exist, the script executes and creates schema while setting its owner to "dbo" (the database owner)
- **Step 3:** Repeat thesame logic to check and create other schemas "stg" and "f"

### **2. Dim Table Creation**
The following tables were created
-  Area
-  Item
-  Calendar

#### Code example using the "insert code" method

```sql
/****************************************************************************/
/*********************   AREA DIM DDL   *************************************/
/***************************************************************************/

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Area')
BEGIN
CREATE TABLE dim.Area(
	Area_Code bigint NOT NULL,
	Area nvarchar(max) NOT NULL
	)
	;

	ALTER TABLE dim.Area
	ADD CONSTRAINT pk_Area PRIMARY KEY(Area_Code);

END;

/****************************************************************************/
/*********************  ITEM DIM DDL   *************************************/
/**************************************************************************/
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Item')
BEGIN
CREATE TABLE dim.Item(
	ItemCode bigint NOT NULL,
	Item nvarchar(max) NOT NULL
	);

	ALTER TABLE dim.Item
	ADD CONSTRAINT pk_Item PRIMARY KEY(ItemCode);
END;

/****************************************************************************/
/*********************  CALENDAR DIM DDL   *************************************/
/*******************************************************************************/
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Calendar')
BEGIN
    CREATE TABLE dim.Calendar (
        pkCalendarID INT NOT NULL,
        Year INT NOT NULL
    );
      ALTER TABLE dim.Calendar
    ADD CONSTRAINT PK_Calendar PRIMARY KEY (pkCalendarID);
     ALTER TABLE dim.Calendar
    ADD CONSTRAINT UC_Calendar UNIQUE (Year);
END;

```

### **Explanation of Dim Tables Creation**
**Area and Item Tables**
- **Step 1:** Checking for the existence of the tables in the schema "dim"
- **Step 2:** If the tables do not exist, system view called INFORMATION_SCHEMA.TABLES - was used to look for tables names "Area" and "Item" in the schema "dim"
- **Step 3:** Tables names dim.Area and dim.Item were created with two columns respectively (Area_code, Area) and (ItemCode, Item). "Area_Code" and "ItemCode" colunms are bigint columns which can not be null while "Area" and "Item" columns are nvarchar(max) which can not also be null.
- **Step 4:** Primary Key Constraints were defined for both tables i.e Area_Code and ItemCode in order to ensure that the values are unique with a unique identifier.

**Calendar Table**
- **Step 1:** Checking for the existence of dim.calendar in the schema "dim"
- **Step 2:** If the table does not exist, system view called INFORMATION_SCHEMA.TABLES - was used to look for table names "Calendar" in the schema "dim"
-  **Step 3:** "pkCalendarID" was taken as the primary key column that uniquely idenfies each row and is defined as "not null" to ensure there is a value in it. The "year" column is set to "integer" and "not null" so as to guarantee the column  has a valid data
- **Step 4:** Primary key constraint was created to ensure that "pkCalendarID" uniquely idenfies every record and automatically create a clustered index in the column.
 **Step 5:** Unique constraint was created to ensure that each year appears only once in the table.

 ### **3. Fact (f) Table Creation**
The following tables were created
-  fact

#### Code example using the "insert code" method

```sql
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'f' AND TABLE_NAME = 'fact')
BEGIN
CREATE TABLE f.fact(
	Area_Code bigint NOT NULL,
	ItemCode int NOT NULL,
	Year int NOT NULL,
	hg_ha_Yield int NULL,
	Average_rainfall int NULL,
	pesticides int NOT NULL,
	avg_temp int NULL,
	Min_Rainfall_Reqd int NULL,
	Weather_Index_Insurance nvarchar(10)
);
ALTER TABLE f.fact
ADD CONSTRAINT  FK_Item FOREIGN KEY(ItemCode) REFERENCES dim.item(ItemCode);
ALTER TABLE f.fact
ADD CONSTRAINT FK_Area FOREIGN KEY (Area_Code) REFERENCES dim.Area(Area_Code);
ALTER TABLE f.fact
ADD CONSTRAINT FK_Year FOREIGN KEY (Year) REFERENCES dim.Calendar(Year);
END;
```

### **Explanation of Fact Table Creation**
- **Step 1:** Checking for the existence of the tables in the schema "f"
- **Step 2:** If the tables do not exist, system view called INFORMATION_SCHEMA.TABLES - was used to look for tables name "fact" in the schema "f"
- **Step 3:** Fact table structure
    -  Area_Code: Links to the dim.Area table (foreign key).
    - ItemCode: Links to the dim.item table (foreign key).
    -  Year: Represents the year of the recorded fact, linking to dim.Calendar.
    - hg_ha_Yield: (nullable) The crop yield in hectograms per hectare.
    - Average_rainfall: (nullable) The average rainfall for the year/area.
    - pesticides: The total pesticides used; this is a mandatory field.
    - avg_temp: (nullable) The average temperature for the year/area.
    - Min_Rainfall_Reqd: (nullable) The minimum rainfall required for optimal crop growth.
    - Weather_Index_Insurance: Whether a weather insurance claim was made; values are "Claim" or "No Claim"
- **Step 4:** Foreign keys were created to establish a relationship between the fact table and the dim tables

 ### **3. Project ERD**
![ERD](./SamuelBIProject.png)
 
 
 ### **4. Data Loading for Test (Seed Data)**

 Data were loaded into the following tables from stg
-  Area
-  Item
-  Calendar
- Fact

```sql
/****************************************************************************/
/*********************  DIM AREA LOADER  ***********************************/
/*******************************************************************************/
INSERT INTO dim.Area(Area_Code, Area)
SELECT psda.Area_Code
		  ,psda.Area
FROM Project.stg.dim_Area psda
WHERE NOT EXISTS (
SELECT 1
FROM dim.Area da
WHERE da.Area_Code=psda.Area_Code)
;

/****************************************************************************/
/*********************  DIM ITEM LOADER  ***********************************/
/*******************************************************************************/
INSERT INTO dim.Item(ItemCode, Item)
SELECT psdi.ItemCode
		  ,psdi.Item
FROM Project.stg.dim_Item psdi
WHERE NOT EXISTS (
SELECT 1
FROM dim.Item di
WHERE di.ItemCode=psdi.ItemCode)
;

/****************************************************************************/
/*********************  DIM CALEDAR LOADER  ***********************************/
/*******************************************************************************/

DECLARE @Year INT;
-- Loop through the years from 1990 to 2013
SET @Year = 1990;

WHILE @Year <= 2013
BEGIN
    -- Insert the Year into dim.Calendar table only if it doesn't already exist
    INSERT INTO dim.Calendar (pkCalendarID, [Year])
    SELECT 
        (@Year - 1990) * 10000 + 1 AS pkCalendarID,  -- Generate pkCalendarID
        @Year AS [Year]                              -- Insert the Year
    WHERE NOT EXISTS (
        SELECT 1
        FROM dim.Calendar dc
        WHERE dc.[Year] = @Year
    );

    -- Move to the next year
    SET @Year = @Year + 1;
END;


/****************************************************************************/
/*********************  FACT TABLE LOADER  ***********************************/
/*******************************************************************************/

INSERT INTO f.fact(Area_Code,ItemCode,	Year, hg_ha_Yield, Average_rainfall,pesticides,	avg_temp, Min_Rainfall_Reqd, Weather_Index_Insurance)

	INSERT INTO f.fact(Area_Code,ItemCode,	Year, hg_ha_Yield, Average_rainfall,pesticides,	avg_temp, Min_Rainfall_Reqd, Weather_Index_Insurance)

	SELECT sq.Area_Code as f_Area_Code,
		sq.ItemCode as f_ItemCode,
		sq.Year as f_Year,
		sq.[hg/ha_yield] as f_Yield,
		sq.average_rain_fall_mm_per_year as f_Avg_rainfall,
		sq.pesticides_tonnes as f_pesticides,
		sq.avg_temp as f_avg_temp,
		sq.[Min_Rainfall-Reqd] as f_min_rainfall_required,
		sq.Weather_Index_Insurance as f_WI_Insurance
	FROM stg.qfact sq
	WHERE NOT EXISTS (
	SELECT 1
	FROM f.fact ff
	WHERE sq.ItemCode = ff.ItemCode
	AND sq.Area_Code = ff.Area_Code
	AND sq.Year	= ff.Year
	);

Go
```

### **Explanation of Data Loading Code**
- **AREA LOADER**
- **Step 1:** Inserted data into dim.Area table specifying that Area_Code and Area will recieve data from the dim.Area table in the project.stg schema aliased 'pdsa'
- **Step 2:** The WHERE NOT EXISTS clause means only the rows where the Area_Code from psda (staging table) do not already exist in the dim.Area table should be inserted. The SELECT 1 is checking for the presence of any rows where the Area_Code matches. If no match is found, the psda.Area_Code is inserted into dim.Area."

- **ITEM LOADER**
- **Step 1:** Inserted data into dim.item table specifying that ItemCode and item will recieve data from the dim.Area table in the project.stg schema aliased 'pdsa'
- **Step 2:** The WHERE NOT EXISTS clause means only the rows where the ItemCode from psda (staging table) do not already exist in the dim.Area table should be inserted. The SELECT 1 is checking for the presence of any rows where the ItemCode matches. If no match is found, the psda.ItemCode is inserted into dim.Item."

- **CALENDAR LOADER**

The SQL script inserts a year into the dim.Calendar table for the years between 1990 and 2013 (inclusive), but only if that year doesn't already exist in the table
- **Step 1:** Declaring a variable integer @year to store the year value in the loop
- **Step 2** Initializing the @year variable to 1990 which is the loop starting year
- **Step 3** A while loop was intiated to run as long as @year is less than or equal to 2013
- **Step 4** A new record with "pkCalendarID'ensuring that it is unique for the year is inserted. it is unique for each and begins at 1 for 1990 and increasing by 1 for each year.
- **Step 5** "where not exists" clause ensures that a year is only inserted if it doesn't already exist in the dim.Calendar table
- **Step 6** After each instance, the year is increased by 1 moving to the next year.

**FACT LOADER**
- **Step 1:** INSERT INTO f.fact specifies the target table and the columns into which the data will be inserted: Area_Code, ItemCode, Year, hg_ha_Yield, Average_rainfall, pesticides, avg_temp, Min_Rainfall_Reqd, and Weather_Index_Insurance.

- **Step 2:** SELECT Clause selects the data from the stg.qfact source table and renames the columns for insertion into f.fact.
For example, sq.Area_Code AS f_Area_Code selects the Area_Code from stg.qfact and aliases it as f_Area_Code to match the target table structure in f.fact.

- **Step 3:** WHERE NOT EXISTS ensures that only records from stg.qfact that do not already exist in f.fact (based on ItemCode, Area_Code and Year) are inserted. This prevents inserting duplicate rows into the f.fact table.
